In [1]:
import re
import json
import sys
import os
import pickle
import shutil
import random

In [2]:
#Root = "/datasets/sagarj/vineData/Dataset/"
Root = "/datasets/sagarj/UnPopular2016_senti/"
videoDir = Root+"Videos/"
postDir = Root+"Posts/"

#sampleRoot = "/datasets/sagarj/sampledVidsPop/"
sampleRoot = "/datasets/sagarj/sampledVidsUnpop/"
sampledVids = sampleRoot+"Videos/"
sampledPosts = sampleRoot+"Posts/"

SampleNumbers = 200

In [3]:
def readJson(path):
    f = open(path)
    data = json.loads(f.read())
    return data

def getPosts(postsDir):
    crawledPosts = os.listdir(postsDir)
    posts = []
    for post in crawledPosts:
        record = readJson(postsDir + post)
        p = record['data']
        if isinstance(p,dict):
            posts.append(p['records'][0])
    return posts

def getMappingDict(postList):
    mapping = dict()
    for p in postList:
        postId = p['postId']
        vidName = p['videoUrl'].split('/')[5].split('?')[0]
        mapping[postId] = vidName
    return mapping


def copyFile(src, dest):
    if not os.path.exists(src): 
        return
    try:
        shutil.copy(src, dest)
    # eg. src and dest are the same file
    except shutil.Error as e:
        print('Error: %s' % e)
    # eg. source or destination doesn't exist
    except IOError as e:
        print('Error: %s' % e.strerror)

In [4]:
postList = getPosts(postDir)
mappingDict = getMappingDict(postList)

In [5]:
print mappingDict.keys()[100], mappingDict[mappingDict.keys()[100]]

1385515636687552512 93231F35C81385515581339561984_5ec0c4bfbae.31.0.0951E4D7-7B1D-4441-A5CF-FB37B65105DD.mp4


In [6]:
sampledKeys = random.sample(mappingDict.keys() , SampleNumbers)

In [7]:
for key in sampledKeys: 
    jsonFile = postDir + str(key) + ".json"
    videoFile = videoDir + mappingDict[key]
    if os.path.exists(jsonFile): 
        print "Json Present: " + jsonFile
        copyFile(jsonFile , sampledPosts)
    if os.path.exists(videoFile): 
        print "Video Present: " + videoFile
        copyFile(videoFile , sampledVids)

Json Present: /datasets/sagarj/UnPopular2016_senti/Posts/1383517804267216896.json
Video Present: /datasets/sagarj/UnPopular2016_senti/Videos/05EAE9922C1383517671081074688_SW_WEBM_14728305282262df2309d05.mp4
Json Present: /datasets/sagarj/UnPopular2016_senti/Posts/1379641548110778368.json
Video Present: /datasets/sagarj/UnPopular2016_senti/Videos/A76F8BD9D81379641545774559232_5da091d796c.31.0.D73A482A-8C70-4E3C-856E-7F68260C6CCA.mp4
Json Present: /datasets/sagarj/UnPopular2016_senti/Posts/1384762039696039936.json
Video Present: /datasets/sagarj/UnPopular2016_senti/Videos/93EDE674CE1384761977242804224_58071526133.33.0.9E46AF5C-28B7-4C92-9177-5B04AFCD83F6.mp4
Json Present: /datasets/sagarj/UnPopular2016_senti/Posts/1384872707916820480.json
Video Present: /datasets/sagarj/UnPopular2016_senti/Videos/27DABFC8A01384872476668211200_57114ef567d.33.0.91499A08-CC74-415C-87BD-E82B1631A507.mp4
Json Present: /datasets/sagarj/UnPopular2016_senti/Posts/1380726953065238528.json
Video Present: /datasets